In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)



In [3]:
import numpy as np
from skimage import io, color, transform
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import joblib
from keras.applications import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Set the path to the dataset
train_data_path = '/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/train/'

num_classes = 6


# Creating an empty lists to store data and labels
data = []
labels = []

def load_and_preprocess_images(folder_path, label):
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            image = io.imread(os.path.join(folder_path, filename))
            # Convert grayscale to RGB by duplicating the single channel
            if image.shape[2] == 1:
                image = np.stack((image,) * 3, axis=-1)
            # Preprocess the image
            image = transform.resize(image, (100, 100, 3))
            data.append(image)
            labels.append(label)

# Load and preprocess images from the "train" folder
load_and_preprocess_images(os.path.join(train_data_path, 'freshapples'), 'freshapples')
load_and_preprocess_images(os.path.join(train_data_path, 'freshbanana'), 'freshbanana')
load_and_preprocess_images(os.path.join(train_data_path, 'freshoranges'), 'freshoranges')
load_and_preprocess_images(os.path.join(train_data_path, 'rottenapples'), 'rottenapples')
load_and_preprocess_images(os.path.join(train_data_path, 'rottenbanana'), 'rottenbanana')
load_and_preprocess_images(os.path.join(train_data_path, 'rottenoranges'), 'rottenoranges')

# Encoding the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Converting data and labels to NumPy arrays
data = np.array(data, dtype=np.float32)  # Ensuring data in float32
labels = np.array(labels)



# Spliting the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(data, labels, test_size=0.2, random_state=42)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)

# Creating a data generator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Loaded the pre-trained DenseNet121 model without top classification layers
base_model = DenseNet121(include_top=False, input_shape=(100, 100, 3), weights=None)

# Adding custom classification layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Creating the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compiling the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model 
model.fit(X_train, y_train, batch_size=32, steps_per_epoch=len(X_train) / 60, epochs=5)

# Evaluating the accuracy on the validation dataset
y_valid_pred = model.predict(X_valid)
y_valid_pred = np.argmax(y_valid_pred, axis=1)
y_valid = np.argmax(y_valid, axis=1)  # If one-hot encoding is used for labels
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
print(f'Validation Accuracy: {valid_accuracy * 100:.2f}%')

# Saving the trained model 
model.save('densenet_fruit_classifierr.h5')


/opt/conda/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
145/145 [==============================] - 284s 2s/step - loss: 0.7813 - accuracy: 0.7322
Epoch 2/5
145/145 [==============================] - 273s 2s/step - loss: 0.5531 - accuracy: 0.8112
Epoch 3/5
145/145 [==============================] - 265s 2s/step - loss: 0.4588 - accuracy: 0.8414
Epoch 4/5
145/145 [==============================] - 272s 2s/step - loss: 0.4122 - accuracy: 0.8548
Epoch 5/5
69/69 [==============================] - 28s 388ms/step
Validation Accuracy: 77.95%
